In [ ]:
# Energy Plus standard simulation
import opyplus as op
import os
import pandas as pd

In [ ]:
# Get paths of the idf and epw files
eplus_dir_path = op.get_eplus_base_dir_path((9, 6, 0))
expidf_path = os.path.join("P8_files", "P8_new.idf")
epw_path = os.path.join("P8_files", "Zürich-hour.epw")

In [ ]:
# Load EnergyPlus Model
epm = op.Epm.from_idf(expidf_path)

In [ ]:
# Define variable material - wall, ground and roof

variable_insulation_wall = epm.Material.one("variable_insulation_wall")
variable_insulation_ground = epm.Material.one("variable_insulation_ground")
variable_insulation_roof = epm.Material.one("variable_insulation_roof")

print(variable_insulation_wall)
print(variable_insulation_ground)
print(variable_insulation_roof)

In [ ]:
# Import material properties form python file
from materials import materials

for name, material in materials.items():
    print(name)
    print(material)

In [ ]:
# Change the variable materials and run the simulation for each material

for name, material in materials.items():
    variable_insulation_wall.thickness = material.thickness_wall
    variable_insulation_ground.thickness = material.thickness_ground
    variable_insulation_roof.thickness = material.thickness_roof
    variable_insulation_wall.conductivity = material.conductivity
    variable_insulation_ground.conductivity = material.conductivity
    variable_insulation_roof.conductivity = material.conductivity
    variable_insulation_wall.density = material.density
    variable_insulation_ground.density = material.density
    variable_insulation_roof.density = material.density
    variable_insulation_wall.specific_heat = material.specific_heat
    variable_insulation_ground.specific_heat = material.specific_heat
    variable_insulation_roof.specific_heat = material.specific_heat
    
    # Run the simulation
    folder_name = material.name + str(material.u_value)
    simulation_results = op.simulate(expidf_path, epw_path, folder_name)
    
    # Save the new idf file
    folder_name_idf = material.name + str(material.u_value) + ".idf"
    epm.save(folder_name+"/"+folder_name_idf)
 
    # Print the summary table on site energy and envelope exterior
    site_source_energy = simulation_results.get_out_summary_table().get_table_df(
    report_key='Annual Building Utility Performance Summary_Entire Facility',
    table_report='Site and Source Energy',
    )
    print(site_source_energy)
    
    envelope_summary_exterior = simulation_results.get_out_summary_table().get_table_df(
    report_key='Envelope Summary_Entire Facility',
    table_report='Opaque Exterior',
    )
    print(envelope_summary_exterior)
    
    # Calculcate volume of used materials
    surface_variable_wall = 813.96
    surface_variable_ground = 187.39
    surface_variable_roof = 220.51
    volume_variable_wall = surface_variable_wall*material.thickness_wall
    volume_variable_ground = surface_variable_wall*material.thickness_ground
    volume_variable_roof = surface_variable_wall*material.thickness_roof
    
    # Calculate embodied emissions
    embodied_emissions_wall = volume_variable_wall*material.embodied_emissions
    embodied_emissions_ground = volume_variable_wall*material.embodied_emissions
    embodied_emissions_roof = volume_variable_wall*material.embodied_emissions
    material.embodied_emissions_total = embodied_emissions_wall + embodied_emissions_ground + embodied_emissions_roof

    # Calculate price
    price_wall = volume_variable_wall*material.price_wall
    price_ground = volume_variable_wall*material.price_ground
    price_roof = volume_variable_wall*material.price_roof
    material.price_total = price_wall + price_ground + price_roof
    

In [ ]:
# Print status of last simulation
print(f"status: {simulation_results.get_status()}\n")
print(f"Eplus .err file:\n{simulation_results.get_out_err().get_content()}")

In [ ]:
for name, material in materials.items():
    print(name)
    print('Total price: ', material.price_total)
    print('Total embodied emissions: ', material.embodied_emissions_total)